# Text clean up for all inactive ingredients

In [21]:

import numpy as np
import pandas as pd
import string
import copy
import os
import re
import json
from collections import defaultdict

## Import in excipients Data

In [22]:
inactives_path = r"C:\Users\nvu\Desktop\Data_Projects\IIR_OCOMM.csv"

alphabet_ls = list(string.ascii_uppercase)

#read inactives into pandas
inactives_df = pd.read_csv(inactives_path)
# get unique_ingredients
unique_ingredients = inactives_df['INGREDIENT_NAME'].unique()

## SOlutions 1: Build dictionary to look up Excipient
### many complication = > mainly that there are many complicated exipients with long words and building a dictionary look up is complicated

In [3]:
#convert list into nested Dictionary
def dictify(lst):
    if not lst:
        return None

    sub_dict = dictify(lst[1:])
    return {value: copy.deepcopy(sub_dict) for value in lst[0]}

#conver chemical names into dictionary format
def split_words(ingredient_str):
    spec_chars = r"[].,'"
    for i in spec_chars:
        ingredient_str = ingredient_str.replace(i, "")
    words = ingredient_str.split(" ")
    result_out = []
    for w in words:
        result_out.append([w])
    return dictify(result_out)


In [4]:
split_words("FD&C RED NO. 40")

{'FD&C': {'RED': {'NO': {'40': None}}}}

In [20]:
max_len = 0
for test_u in unique_ingredients:
    spec_chars = r"[].,'"
    for i in spec_chars:
        test_u = test_u.replace(i, "")
    tem_max_len = len(test_u.split(" "))
    if tem_max_len > max_len:
        max_len = tem_max_len
        
print(max_len)
    

10


In [5]:
dict_list = [split_words(u) for u in unique_ingredients]

excipients_dict = defaultdict(list)
#build look up dictionary
for d in dict_list:
    name_k = []
    for key, val in d.items():
        excipients_dict[key].append(val)




In [6]:
for i in excipients_dict["ZINC"]:
    try:
        if 'OXIDE' in i.keys():
            print("ZINC OXIDE")
    except:
        continue

ZINC OXIDE


In [7]:
#Dump excipientNames to JSON to use in future applications

with open("excipient_names_dict.json", "w") as outfile:
    json.dump(excipients_dict, outfile)

In [8]:
len(unique_ingredients)

1786

## Import in openFDA data

In [9]:
with open(r'C:\Users\nvu\Desktop\Data_Projects\OpenFDA\excipient_names_dict.json') as json_file:
    exp_dict = json.load(json_file)

data_openFDA = pd.read_csv(r"C:\Users\nvu\Desktop\Data_Projects\Drug_Prices\OpenFDA_LabelExplorer\10Aug21_OpenFDA.csv")

data_openFDA.columns

Index(['Unnamed: 0', 'API', 'Description', 'SPL', 'package_ndc', 'unii',
       'manufacturer_name', 'pharm_class_pe', 'product_type', 'brand_name',
       'substance_name', 'pharm_class_cs', 'pharm_class_epc', 'route'],
      dtype='object')

In [10]:
unclean_ingredients = data_openFDA.SPL.values
unclean_ingredients_copy = unclean_ingredients[:100].copy()


In [11]:
x = unclean_ingredients_copy[3]

In [12]:
for e in exp_dict["ZINC"]:
    try:
        print(list(e.keys()))
    except:
        continue

['ACETATE']
['CARBONATE']
['CHLORIDE']
['OXIDE']
['STEARATE']
['SULFATE']


## Solution 1 Continue: Using the multiple nested level of dictionary to look up excipients. May work but very very cummbersome. With longer exicpients name, this will not work well as we have to traverse thru multiple leaf in the nested level. Can be prone to mistake
### Abandon approach to try Jaccard

In [ ]:
#write function to parse out ingrdients
def clean_up_SPL(spl_str_in):
    spl_str = spl_str_in.upper()
    p = re.compile(r"([a-zA-Z]+)", re.IGNORECASE+re.M)
    words  = p.findall(spl_str)
    return " ".join(set(words))

def get_parsed_ingredients(spl_str_in, excipient_dictionary):
    exp_dict = excipient_dictionary
    clean_str = clean_up_SPL(spl_str_in)
    clean_str_ls = clean_str.split(" ")
    
    cleaned_excipients = []
    temp_exc = ""
    for exc in clean_str_ls:
        if exc in exp_dict: #check if word is in exciipiend dictionary
            cleaned_excipients.append(exc) # if in dict, add first word to cleaned excipient list
            temp_2nd_words_ls = [] # temp list to store correct 2nd word
            for e in exp_dict[exc]: # loop thru the embedded list of the 1st word
                try:
                   temp_2nd_words_ls.extend(list(e.keys())[0]) # take all possible 2nd word and add to temp list
                except:
                    continue
            
                for temp2 in temp_2nd_words_ls: # loop thru the temp list that stored 
                    if temp2 in clean_str:
                        cleaned_excipients.append(temp2)
                        try:
                            for v in e[temp2]:
                                if v in clean_str:
                                    pass
                        except:
                            continue
                        
    

In [ ]:
clean_up_SPL(x)

In [ ]:

#define function to look up exipient from a dict
def look_up_exc(exc_dictionary, spl_str):
    excipients_list = []
 

In [ ]:
exp_dict["ZINC"]
        
        
        

In [ ]:
exp_dict["ZINC"][1].keys()

In [13]:
exp_dict

{'ALPHA-TERPINEOL': [None],
 'ALPHA-TOCOPHEROL': [None,
  {'ACETATE': None},
  {'ACETATE': {'DL-': None}},
  {'DL-': None}],
 '1-(PHENYLAZO)-2-NAPHTHYLAMINE': [None],
 '126-HEXANETRIOL': [None],
 '12-DISTEAROYL-SN-GLYCERO-3-PHOSPHOCHOLINE': [None],
 '14-SORBITAN': [None],
 '2-(4-STYRYL-3-SULFOPHENOL)-6-METHOXYBENZOTRIAZOLE': [None],
 'ACACIA': [None, {'SYRUP': None}],
 'ACESULFAME': [None, {'POTASSIUM': None}],
 'ACETIC': [{'ACID': None}, {'ANHYDRIDE': None}],
 'ACETYLTRIBUTYL': [{'CITRATE': None}],
 'ACRYLATES': [{'COPOLYMER': None}],
 'ACRYL-EZE': [{'93018509': {'WHITE': None}}, {'93084720': {'PINK': None}}],
 'ACRYLIC': [{'ACID-ISOOCTYL': {'ACRYLATE': {'COPOLYMER': None}}},
  {'ADHESIVE': {'788': None}}],
 'ADCOTE': [{'72A103': None}],
 'ADHESIVE': [{'TAPE': None}],
 'ADIPIC': [{'ACID': None}],
 'ADVANTIA': [{'PRIME': {'190100BA01': {'WHITE': None}}}],
 'AGAR': [None],
 'ALBUMIN': [{'HUMAN': None}],
 'ALBUMINS': [None],
 'ALCLOXA': [None],
 'ALCOHOL': [None],
 'ALFADEX': [None],
 'A

## Solution 2: Thought of this late after wasting sometime: Jaccard Simmilarity will work here as a more elegant solution:

* Create a 1 level dictionary to look up exicpient by 1st word.
* 2nd level of dictioonary will be a tuple of possible excipients with the same 1st word
* Looking up algorithm will take in the unparsed string of excipient, loop thru each word to locate the key in excipient dictionary for the 1st word
* If 1st word located, check for jaccard similarity between tupple and long string list.


In [44]:

# Build a 1 level dictionary with first word and following list of words. 
# 
# Define a function to parse excipients, Options exist to count numbers
def words_parser(string_in, number = False):
    if number is False:
        p = re.compile(r"([a-zA-Z]+)", re.IGNORECASE+re.M)
    else:
        p = re.compile(r"([a-zA-Z0-9]+)", re.IGNORECASE+re.M)
    words  = p.findall(string_in.upper())
    return words

def build_dictionary(list_to_parse):
    dic_store = defaultdict(list)
    for i in list_to_parse:
        dic_store[words_parser(i)[0]].append(" ".join(words_parser(i, number=True)))

    return dic_store

    

In [141]:

# Jaccard difference implenetation
def get_jaccard_sim_words(excipient_word1, unparsed_long_word):
    w1_set = set(excipient_word1.split())
    w2_set = set(unparsed_long_word.split())
    
    similar_elements = w1_set.intersection(w2_set)
    
    if len(similar_elements) == len(w1_set):
        return excipient_word1
    else:
        return ""
    

def excipients_jaccard(unparsed_long_word, dictionary_lookup):
    excipients = []
    unparsed_long_word_clean = words_parser(unparsed_long_word)
    for word in unparsed_long_word_clean:
        if len(word)>2:
            if dictionary_lookup.get(word) is not None:
                look_up_list = dictionary_lookup.get(word)
                for word2 in look_up_list:
                    try:
                        excipients.append(get_jaccard_sim_words(word2, " ".join(words_parser(unparsed_long_word, number=True))))
                    except:
                        continue
            else:
                excipients.append(word.lower())
   
    return ",".join(set([e for e in excipients if ((e not in "to and no") and (e!=""))]))
                    
        
 

#### Small testing section

In [110]:



unparsed_test = "ALPHA TOCOPHEROL ACETATE PHOSPHOCHOLINE POLYSORBATE DISTEAROYL SN GLYCERO 3 WATER POOP JUMK JUNKS MORE JUNLKS SHIT FART PEE BURP LOUD BURP"

test_ingre = unique_ingredients[0:10]
test_ingre = np.append(test_ingre, "POOP")
test_ingre = np.append(test_ingre, "WET PEE")
test_ingre = np.append(test_ingre, "APPLE FART")
test_ingre = np.append(test_ingre, "LOUD BURP")
test_ingre = np.append(test_ingre, "WET POOP")
test_ingre = np.append(test_ingre, "LOUD POOP")



test_dict_pihip = build_dictionary(test_ingre)


In [111]:
for i in test_ingre:
    print(i)

.ALPHA.-TERPINEOL
.ALPHA.-TOCOPHEROL
.ALPHA.-TOCOPHEROL ACETATE
.ALPHA.-TOCOPHEROL ACETATE, DL-
.ALPHA.-TOCOPHEROL, DL-
1-(PHENYLAZO)-2-NAPHTHYLAMINE
1,2,6-HEXANETRIOL
1,2-DISTEAROYL-SN-GLYCERO-3-PHOSPHOCHOLINE
1,4-SORBITAN
2-(4-STYRYL-3-SULFOPHENOL)-6-METHOXYBENZOTRIAZOLE
POOP
WET PEE
APPLE FART
LOUD BURP
WET POOP
LOUD POOP


In [112]:
test_dict_pihip

defaultdict(list,
            {'ALPHA': ['ALPHA TERPINEOL',
              'ALPHA TOCOPHEROL',
              'ALPHA TOCOPHEROL ACETATE',
              'ALPHA TOCOPHEROL ACETATE DL',
              'ALPHA TOCOPHEROL DL'],
             'PHENYLAZO': ['1 PHENYLAZO 2 NAPHTHYLAMINE'],
             'HEXANETRIOL': ['1 2 6 HEXANETRIOL'],
             'DISTEAROYL': ['1 2 DISTEAROYL SN GLYCERO 3 PHOSPHOCHOLINE'],
             'SORBITAN': ['1 4 SORBITAN'],
             'STYRYL': ['2 4 STYRYL 3 SULFOPHENOL 6 METHOXYBENZOTRIAZOLE'],
             'POOP': ['POOP'],
             'WET': ['WET PEE', 'WET POOP'],
             'APPLE': ['APPLE FART'],
             'LOUD': ['LOUD BURP', 'LOUD POOP']})

In [113]:
excipients_jaccard(unparsed_long_word=unparsed_test, dictionary_lookup=test_dict_pihip)

'POOP,BURP,ALPHA TOCOPHEROL ACETATE,FART,TOCOPHEROL,WATER,GLYCERO,LOUD POOP,PEE,SN,LOUD BURP,POLYSORBATE,JUNKS,MORE,JUMK,SHIT,PHOSPHOCHOLINE,ALPHA TOCOPHEROL,JUNLKS,ACETATE'

## Implement excipients parse to real OpenFDA Data

In [142]:

#Lookup Dictionary For all Inactives:
excipients_dictionary = build_dictionary(unique_ingredients)

#Parse out excipients for each drug
parsed_excipients = [excipients_jaccard(u, dictionary_lookup=excipients_dictionary) for u in unclean_ingredients] #takes 250 sec to complete 37974 rows


In [143]:
data_openFDA["Ingredients_clean_csv"] = data_openFDA["SPL"].apply(lambda x: excipients_jaccard(x, dictionary_lookup=excipients_dictionary))  #takes 236 sec to complete 37974 rows

In [144]:
data_openFDA.head(3)

,Unnamed: 0,API,Description,SPL,package_ndc,unii,manufacturer_name,pharm_class_pe,product_type,brand_name,substance_name,pharm_class_cs,pharm_class_epc,route,Ingredients_clean_csv
0,0,acamprosate,11 DESCRIPTION Acamprosate calcium is supplied...,acamprosate calcium acamprosate calcium ACAMPR...,"['70771-1057-3', '70771-1057-9', '70771-1057-1...",['No_Info'],['Cadila Healthcare Limited'],['No_Info'],['HUMAN PRESCRIPTION DRUG'],['acamprosate calcium'],['ACAMPROSATE CALCIUM'],['No_Info'],['No_Info'],['ORAL'],"glycolate,MICROCRYSTALLINE CELLULOSE,potato,SI..."
1,1,acamprosate,11 DESCRIPTION Acamprosate calcium delayed-rel...,Acamprosate Calcium Acamprosate Calcium ACAMPR...,"['68462-435-18', '68462-435-11']",['No_Info'],"['Glenmark Pharmaceuticals Inc., USA']",['No_Info'],['HUMAN PRESCRIPTION DRUG'],['Acamprosate Calcium'],['ACAMPROSATE CALCIUM'],['No_Info'],['No_Info'],['ORAL'],"glycolate,METHACRYLIC ACID COPOLYMER,MICROCRYS..."
2,2,acamprosate,11 DESCRIPTION Acamprosate calcium is supplied...,Acamprosate Calcium acamprosate calcium enteri...,['0378-6333-80'],['No_Info'],['Mylan Pharmaceuticals Inc.'],['No_Info'],['HUMAN PRESCRIPTION DRUG'],['Acamprosate Calcium'],['ACAMPROSATE CALCIUM'],['No_Info'],['No_Info'],['ORAL'],"METHACRYLIC ACID COPOLYMER,MICROCRYSTALLINE CE..."


In [145]:
data_openFDA.to_csv(os.path.join(os.getcwd(), "OpenFDA_ingredients_parsed_210823_2.csv"))

In [146]:
graphical_data_base_prep = data_openFDA.loc[:, ["API", "package_ndc", "Ingredients_clean_csv"]]

In [147]:
graphical_data_base_prep.to_csv(os.path.join(os.getcwd(), "OpenFDA_ingredients_parsed_forNEO4J_210823_2.csv"))

In [149]:
data_openFDA.iloc[17403, :]["SPL"]

'Valsartan and Hydrochlorothiazide Valsartan and Hydrochlorothiazide VALSARTAN VALSARTAN HYDROCHLOROTHIAZIDE HYDROCHLOROTHIAZIDE SILICON DIOXIDE CROSPOVIDONE (15 MPA.S AT 5%) HYPROMELLOSE 2910 (5 MPA.S) HYPROMELLOSE 2910 (6 MPA.S) LACTOSE MONOHYDRATE MAGNESIUM STEARATE MICROCRYSTALLINE CELLULOSE POLYETHYLENE GLYCOL 4000 SODIUM LAURYL SULFATE TALC TITANIUM DIOXIDE FERRIC OXIDE YELLOW Ovaloid, Beveled Edge, Biconvex I;65'

In [150]:
data_openFDA.iloc[17403, :]["Ingredients_clean_csv"]

'HYPROMELLOSE 2910 5 MPA S,HYPROMELLOSE 2910 15 MPA S,MICROCRYSTALLINE CELLULOSE,SILICON DIOXIDE,FERRIC OXIDE YELLOW,LACTOSE,MAGNESIUM OXIDE,stearate,HYPROMELLOSE 2910 4000 MPA S,MAGNESIUM SULFATE,LAURYL SULFATE,SODIUM SULFATE,HYPROMELLOSE,valsartan,TALC,beveled,SODIUM CELLULOSE,POLYETHYLENE GLYCOL 4000,mpa,hydrochlorothiazide,SODIUM STEARATE,biconvex,CROSPOVIDONE,LACTOSE MONOHYDRATE CELLULOSE MICROCRYSTALLINE,ovaloid,MAGNESIUM STEARATE,monohydrate,GLYCOL STEARATE,oxide,LACTOSE MONOHYDRATE,HYPROMELLOSE 2910 6 MPA S,SODIUM LAURYL SULFATE,sulfate,TITANIUM DIOXIDE,SILICON,dioxide,edge'

# NOTE TO SELF: Paralelize this function in the future to speed it up.